# Shor's factoring algorithm (step by step)

I don't reccomend to run whole notebook on your laptop.(it may take so much time to be done.)

In [1]:
# import requirements
import array
import math
import fractions
import numpy as np
from qiskit.aqua.algorithms.single_sample.shor import shor
from qiskit.aqua.circuits import FourierTransformCircuits as ftc
from qiskit import QuantumRegister, QuantumCircuit, ClassicalRegister,IBMQ, Aer, execute
np.set_printoptions(linewidth=10000, threshold=np.inf)

Loading your account.  Probably, the project of get_provider method is different from your information.   
Check if it's the same in your account page.
[IBMQ expeirnce](https://quantum-computing.ibm.com/account)

In [17]:
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-keio', group='keio-internal', project='keio-students')
backend = provider.get_backend('ibmq_qasm_simulator')
state_vec = Aer.get_backend('statevector_simulator')

/Users/ryosuke/.local/share/virtualenvs/TERMproject-HGT1Kjoj/lib/python3.7/site-packages/qiskit/providers/ibmq/ibmqfactory.py:181: UserWarning: Credentials are already in use. The existing account in the session will be replaced.
  warnings.warn('Credentials are already in use. The existing '


This function is to print out wavefunction of a circuit.

In [20]:
# Cite: Koch, Daniel, Laura Wessing, and Paul M. Alsing. "Introduction to Coding Quantum Algorithms: A Tutorial Series Using Pyquil." arXiv preprint arXiv:1903.05195 (2019).
import math as m
import scipy as sci

S_simulator = Aer.backends(name='statevector_simulator')[0]
M_simulator = Aer.backends(name='qasm_simulator')[0] 

def Wavefunction( obj , *args, **kwargs):
#Displays the waveftmction of the quantum system 
	if(type(obj) == QuantumCircuit ):
		statevec = execute( obj, S_simulator, shots=1 ).result().get_statevector()
	if(type(obj) == np.ndarray):
		statevec = obj
	sys = False
	NL = False
	dec = 5
	if 'precision' in kwargs:
		dec = int( kwargs['precision'] )
	if 'column' in kwargs:
		NL = kwargs['column']
	if 'systems' in kwargs:
		systems = kwargs['systems']
		sys = True
		last_sys = int(len(systems)-1)
		show_systems = []
		for s_chk in np.arange(len(systems)):
			if( type(systems[s_chk]) != int ):
				raise Exception('systems must be an array of all integers')
		if 'show_systems' in kwargs:
			show_systems = kwargs['show_systems']
			if( len(systems)!= len(show_systems) ):
				raise Exception('systems and show_systems need to be arrays of equal length')
			for ls in np.arange(len(show_systems)):
				if((show_systems[ls] != True) and (show_systems[ls] != False)):
					raise Exception('show_systems must be an array of Truth Values')
				if(show_systems[ls] == True):
					last_sys = int(ls) 
		else:
			for ss in np.arange(len(systems)):
				show_systems.append(True)
	wavefunction = ''
	qubits = int(m.log(len(statevec),2))
	for i in np.arange( int(len(statevec))):
		#print(wavefunction)
		value = round(statevec[i].real, dec) + round(statevec[i].imag, dec) * 1j
		if( (value.real != 0) or (value.imag != 0)):
			state = list(Binary(int(i),int(2**qubits)))
			state.reverse()
			state_str = ''
			#print(state)
			if( sys == True ): #Systems and SharSystems 
				k = 0 
				for s in np.arange(len(systems)):
					if(show_systems[s] == True):
						if(int(s) != last_sys):
							state.insert(int(k + systems[s]), '>|' )
							k = int(k + systems[s] + 1)
						else:
							k = int(k + systems[s]) 
					else:
						for s2 in np.arange(systems[s]):
							del state[int(k)]
			for j in np.arange(len(state)):
				if(type(state[j])!= str):
					state_str = state_str + str(int(state[j])) 
				else:
					state_str = state_str + state[j]
			#print(state_str)
			#print(value)
			if( (value.real != 0) and (value.imag != 0) ):
				if( value.imag > 0):
					wavefunction = wavefunction + str(value.real) + '+' + str(value.imag) + 'j |' + state_str + '>   '
				else:
					wavefunction = wavefunction + str(value.real) + '' + str(value.imag) + 'j |' + state_str +  '>   '
			if( (value.real !=0 ) and (value.imag ==0) ):
				wavefunction = wavefunction  + str(value.real) + '  |' + state_str + '>   '
			if( (value.real == 0) and (value.imag != 0) ):
				wavefunction = wavefunction + str(value.imag)  + 'j |' + state_str + '>   '
			if(NL):
				wavefunction = wavefunction + '\n'
		#print(NL)
	
	#print(wavefunction)
	return wavefunction

def Binary(number,total): 
#Converts a number to binary, right to left LSB 152 153 o
	qubits = int(m.log(total,2))
	N = number
	b_num = np.zeros(qubits)
	for i in np.arange(qubits):
		if( N/((2)**(qubits-i-1)) >= 1 ):
			b_num[i] = 1
			N = N - 2 ** (qubits-i-1)
	B = [] 
	for j in np.arange(len(b_num)):
		B.append(int(b_num[j]))
	return B

Set the two prime numbers $p, q$ and take $N = pq$.   
What we are going to do is calculate $p$ and $q$ only with $N$.

In [4]:
# Two prime number
p = 3
q = 5
#  integer we want to factorize.
N = p * q
# random number which is coprime of N
a = 2
result = {}
result['results'] = {}
result['factors'] = []

We are going to use qiskit example of Shor's algorith. 
source: [get factors](https://github.com/Qiskit/qiskit-aqua/blob/master/qiskit/aqua/algorithms/single_sample/shor/shor.py#L368) (modified)

In [5]:
# verify we got a correct answer.
def get_factors(output_desired, t_upper):
    """
    Apply the continued fractions to find r and the gcd to find the desired factors.
    """
    x_value = int(output_desired, 2)
    if x_value <= 0:
        result['results'][output_desired] = 'x_value is <= 0, there are no continued fractions.'
        return False

    # Calculate T and x/T
    T = pow(2, t_upper)
    x_over_T = x_value / T

    # Cycle in which each iteration corresponds to putting one more term in the
    # calculation of the Continued Fraction (CF) of x/T

    # Initialize the first values according to CF rule
    i = 0
    b = array.array('i')
    t = array.array('f')

    b.append(math.floor(x_over_T))
    t.append(x_over_T - b[i])

    while i >= 0:

        # From the 2nd iteration onwards, calculate the new terms of the CF based
        # on the previous terms as the rule suggests
        if i > 0:
            b.append(math.floor(1 / t[i - 1]))
            t.append((1 / t[i - 1]) - b[i])

        # Calculate the CF using the known terms
        aux = 0
        j = i
        while j > 0:
            aux = 1 / (b[j] + aux)
            j = j - 1

        aux = aux + b[0]

        # Get the denominator from the value obtained
        frac = fractions.Fraction(aux).limit_denominator()
        denominator = frac.denominator

        # Increment i for next iteration
        i = i + 1

        if denominator % 2 == 1:
            if i >= N:
                result['results'][output_desired] = 'unable to find factors after too many attempts.'
                return False
            continue

        # If denominator even, try to get factors of N
        # Get the exponential a^(r/2)
        exponential = 0

        if denominator < 1000:
            exponential = pow(a, denominator / 2)

        # Check if the value is too big or not
        if math.isinf(exponential) or exponential > 1000000000:
            result['results'][output_desired] = 'denominator of continued fraction is too big.'
            return False

        # If the value is not to big (infinity), then get the right values and do the proper gcd()
        putting_plus = int(exponential + 1)
        putting_minus = int(exponential - 1)
        one_factor = math.gcd(putting_plus, N)
        other_factor = math.gcd(putting_minus, N)

        # Check if the factors found are trivial factors or are the desired factors
        if one_factor == 1 or one_factor == N or other_factor == 1 or other_factor == N:
            # Check if the number has already been found, use i-1 because i was already incremented
            if t[i - 1] == 0:
                result['results'][
                    output_desired] = 'the continued fractions found exactly x_final/(2^(2n)).'
                return False
            if i >= N:
                result['results'][output_desired] = 'unable to find factors after too many attempts.'
                return False
        else:
            result['results'][output_desired] = (one_factor, other_factor)
            factors = sorted((one_factor, other_factor))
            if factors not in result['factors']:
                result['factors'].append(factors)
            return True

Construct the circuit to calculate factoring of N.

In [6]:
shor_circuit = shor.Shor(N, a).construct_circuit(measurement=True)
job = execute(shor_circuit, backend=backend)
counts = job.result().get_counts(shor_circuit)

Unexpected exception in keepalive ping task
Traceback (most recent call last):
  File "/Users/ryosuke/.local/share/virtualenvs/TERMproject-HGT1Kjoj/lib/python3.7/site-packages/websockets/protocol.py", line 989, in keepalive_ping
    ping_waiter, self.ping_timeout, loop=self.loop
  File "/usr/local/Cellar/python/3.7.4_1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/asyncio/tasks.py", line 442, in wait_for
    return fut.result()
  File "/usr/local/Cellar/python/3.7.4_1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/asyncio/futures.py", line 178, in result
    raise self._exception
websockets.exceptions.ConnectionClosed: WebSocket connection is closed: code = 1006 (connection closed abnormally [internal]), no reason


After measurement, we need to verify if we got a correct value or not.

In [7]:
# For each simulation result, print proper info to user and try to calculate the factors of N
# the number of qubits
n = np.ceil(np.log2(N))
for output_desired in list(counts.keys()):
    # Get the x_value from the final state qubits
    result['results'][output_desired] = None
    success = get_factors(output_desired, int(2 * n))
    if success:
        print('success!')
        print(result['results'][output_desired])
        print('Two prime numbers are: ', result['results'][output_desired][0], 'and', result['results'][output_desired][1])
        break

success!
(5, 3)
Two prime numbers are:  5 and 3


How huge the circuit is?

In [22]:
print('The number of gates', shor_circuit.count_ops())
print('Depth of circuit', shor_circuit.depth())

The number of gates OrderedDict([('u1', 18644), ('cx', 14276), ('u2', 1456), ('u3', 129), ('cswap', 32), ('measure', 8)])
Depth of circuit 17413


Source code of shor's algorithm.   
[qiskit aqua Shor's algorithm](https://qiskit.org/documentation/_modules/qiskit/aqua/algorithms/single_sample/shor/shor.html#Shor)

## Shor's algorithm

In [13]:
def _get_angles(a):
    """
    Calculate the array of angles to be used in the addition in Fourier Space
    """
    s = bin(int(a))[2:].zfill(n + 1)
    angles = np.zeros([n + 1])
    for i in range(0, n + 1):
        for j in range(i, n + 1):
            if s[j] == '1':
                angles[n - i] += math.pow(2, -(j - i))
        angles[n - i] *= np.pi
    return angles

def _phi_add(circuit, q, inverse=False):
    """
    Creation of the circuit that performs addition by a in Fourier Space
    Can also be used for subtraction by setting the parameter inverse=True
    """
    angle = _get_angles(N)
    for i in range(0, n + 1):
        circuit.u1(-angle[i] if inverse else angle[i], q[i])

def _controlled_phi_add(circuit, q, ctl, inverse=False):
    """
    Single controlled version of the _phi_add circuit
    """
    angles = _get_angles(N)
    for i in range(0, n + 1):
        angle = (-angles[i] if inverse else angles[i]) / 2

        circuit.u1(angle, ctl)
        circuit.cx(ctl, q[i])
        circuit.u1(-angle, q[i])
        circuit.cx(ctl, q[i])
        circuit.u1(angle, q[i])

def _controlled_controlled_phi_add(circuit, q, ctl1, ctl2, a, inverse=False):
    """
    Doubly controlled version of the _phi_add circuit
    """
    angle = _get_angles(a)
    for i in range(n + 1):
        # ccphase(circuit, -angle[i] if inverse else angle[i], ctl1, ctl2, q[i])
        circuit.mcu1(-angle[i] if inverse else angle[i], [ctl1, ctl2], q[i])
        
def _controlled_controlled_phi_add_mod_N(circuit, q, ctl1, ctl2, aux, a):
        """
        Circuit that implements doubly controlled modular addition by a
        """
        _controlled_controlled_phi_add(circuit, q, ctl1, ctl2, a)
        _phi_add(circuit, q, inverse=True)
        ftc.construct_circuit(
            circuit=circuit,
            qubits=[q[i] for i in reversed(range(n + 1))],
            do_swaps=False,
            inverse=True
        )
        circuit.cx(q[n], aux)
        ftc.construct_circuit(
            circuit=circuit,
            qubits=[q[i] for i in reversed(range(n + 1))],
            do_swaps=False
        )
        _controlled_phi_add(circuit, q, aux)

        _controlled_controlled_phi_add(circuit, q, ctl1, ctl2, a, inverse=True)
        ftc.construct_circuit(
            circuit=circuit,
            qubits=[q[i] for i in reversed(range(n + 1))],
            do_swaps=False,
            inverse=True
        )
        circuit.u3(np.pi, 0, np.pi, q[n])
        circuit.cx(q[n], aux)
        circuit.u3(np.pi, 0, np.pi, q[n])
        ftc.construct_circuit(
            circuit=circuit,
            qubits=[q[i] for i in reversed(range(n + 1))],
            do_swaps=False
        )
        _controlled_controlled_phi_add(circuit, q, ctl1, ctl2, a)
def _controlled_controlled_phi_add_mod_N_inv(circuit, q, ctl1, ctl2, aux, a):
    """
    Circuit that implements the inverse of doubly controlled modular addition by a
    """
    _controlled_controlled_phi_add(circuit, q, ctl1, ctl2, a, inverse=True)
    ftc.construct_circuit(
        circuit=circuit,
        qubits=[q[i] for i in reversed(range(n + 1))],
        do_swaps=False,
        inverse=True
    )
    circuit.u3(np.pi, 0, np.pi, q[n])
    circuit.cx(q[n], aux)
    circuit.u3(np.pi, 0, np.pi, q[n])
    ftc.construct_circuit(
        circuit=circuit,
        qubits=[q[i] for i in reversed(range(n + 1))],
        do_swaps=False
    )
    _controlled_controlled_phi_add(circuit, q, ctl1, ctl2, a)
    _controlled_phi_add(circuit, q, aux, inverse=True)
    ftc.construct_circuit(
        circuit=circuit,
        qubits=[q[i] for i in reversed(range(n + 1))],
        do_swaps=False,
        inverse=True
    )
    circuit.cx(q[n], aux)
    ftc.construct_circuit(
        circuit=circuit,
        qubits=[q[i] for i in reversed(range(n + 1))],
        do_swaps=False
    )
    _phi_add(circuit, q)
    _controlled_controlled_phi_add(circuit, q, ctl1, ctl2, a, inverse=True)

def _controlled_multiple_mod_N(circuit, ctl, q, aux, a):
        """
        Circuit that implements single controlled modular multiplication by a
        """
        ftc.construct_circuit(
            circuit=circuit,
            qubits=[aux[i] for i in reversed(range(n + 1))],
            do_swaps=False
        )

        for i in range(0, n):
            _controlled_controlled_phi_add_mod_N(
                circuit,
                aux,
                q[i],
                ctl,
                aux[n + 1],
                (2 ** i) * a % N
            )
        ftc.construct_circuit(
            circuit=circuit,
            qubits=[aux[i] for i in reversed(range(n + 1))],
            do_swaps=False,
            inverse=True
        )

        for i in range(0, n):
            circuit.cswap(ctl, q[i], aux[i])

In [9]:
# Get n value used in Shor's algorithm, to know how many qubits are used
n = math.ceil(math.log(N, 2))

# quantum register where the sequential QFT is performed
_up_qreg = QuantumRegister(2 * n, name='up')
# quantum register where the multiplications are made
_down_qreg = QuantumRegister(n, name='down')
# auxilliary quantum register used in addition and multiplication
_aux_qreg = QuantumRegister(n + 2, name='aux')

In [10]:
# Create Quantum Circuit
circuit = QuantumCircuit(_up_qreg, _down_qreg, _aux_qreg)

# Initialize down register to 1 and create maximal superposition in top register
circuit.u2(0, np.pi, _up_qreg)
circuit.u3(np.pi, 0, np.pi, _down_qreg[0])

# Apply the multiplication gates as showed in the report in order to create the exponentiation
for i in range(0, 2 * n):
    _controlled_multiple_mod_N(
        circuit,
        _up_qreg[i],
        _down_qreg,
        _aux_qreg,
        int(pow(a, pow(2, i)))
    )

Wave function we got after modular exponentiation.

In [21]:
# It may take much time to be done...
print('Wave function after modular exponentiation')
print(Wavefunction(circuit))

Wave function after modular exponentiation
0.0625j |000000001000000000>   0.0625j |001000001000100000>   0.0625j |000100001000100000>   0.0625j |000010001000100000>   0.0625j |000001001000100000>   0.0625j |000000101000100000>   0.0625j |000000011000100000>   0.0625j |100000000100100000>   0.0625j |001100000100100000>   0.0625j |001010000100100000>   0.0625j |000110000100100000>   0.0625j |001001000100100000>   0.0625j |000101000100100000>   0.0625j |000011000100100000>   0.0625j |001000100100100000>   0.0625j |000100100100100000>   0.0625j |000010100100100000>   0.0625j |000001100100100000>   0.0625j |001000010100100000>   0.0625j |000100010100100000>   0.0625j |000010010100100000>   0.0625j |000001010100100000>   0.0625j |000000110100100000>   0.0625j |010000000010100000>   0.0625j |111111000110100000>   0.0625j |111110100110100000>   0.0625j |111101100110100000>   0.0625j |111011100110100000>   0.0625j |110111100110100000>   0.0625j |111110010110100000>   0.0625j |111101010110100000

In [23]:
# Apply inverse QFT
ftc.construct_circuit(circuit=circuit, qubits=_up_qreg, do_swaps=True, inverse=True)

Wave function after inverse Quantum Fourier Transform

In [24]:
print('Wave function after Quantum Fourier Transform')
print(Wavefunction(circuit))

Wave function after Quantum Fourier Transform
0.00391j |000000001000000000>   0.00391j |100000001000000000>   0.00391j |010000001000000000>   0.00391j |110000001000000000>   0.00391j |001000001000000000>   0.00391j |101000001000000000>   0.00391j |011000001000000000>   0.00391j |111000001000000000>   0.00391j |000100001000000000>   0.00391j |100100001000000000>   0.00391j |010100001000000000>   0.00391j |110100001000000000>   0.00391j |001100001000000000>   0.00391j |101100001000000000>   0.00391j |011100001000000000>   0.00391j |111100001000000000>   0.00391j |000010001000000000>   0.00391j |100010001000000000>   0.00391j |010010001000000000>   0.00391j |110010001000000000>   0.00391j |001010001000000000>   0.00391j |101010001000000000>   0.00391j |011010001000000000>   0.00391j |111010001000000000>   0.00391j |000110001000000000>   0.00391j |100110001000000000>   0.00391j |010110001000000000>   0.00391j |110110001000000000>   0.00391j |001110001000000000>   0.00391j |1011100010000000

Measure the circuit

In [25]:
up_cqreg = ClassicalRegister(2 * n, name='m')
circuit.add_register(up_cqreg)
circuit.measure(_up_qreg, up_cqreg)

In [ ]:
job = execute(circuit, backend=backend)
counts = job.result().get_counts(circuit)

In [ ]:
# For each simulation result, print proper info to user and try to calculate the factors of N
# the number of qubits
n = np.ceil(np.log2(N))
for output_desired in list(counts.keys()):
    # Get the x_value from the final state qubits
    result['results'][output_desired] = None
    success = get_factors(output_desired, int(2 * n))
    if success:
        print('success!')
        print(result['results'][output_desired])
        print('Two prime numbers are: ', result['results'][output_desired][0], 'and', result['results'][output_desired][1])
        break